
# HeapHHApportionment-2020 (Huntington-Hill via Max-Heap)

**Author:** Jack Henney  
**Course:** CS315 — Algorithms and Data Structures  
**Goal:** Use 2020 U.S. Census data to apportion 435 U.S. House seats among the 50 states using the Huntington-Hill method, implemented with a max-heap. Verify the results against the official `StatesPopulation.csv` file.

**GitHub Repository Link:**  
[Click to view my HeapHHApportionment-2020.ipynb on GitHub](https://github.com/jackhenney/CS315/blob/main/heapH5.ipynb)


In [ ]:

import math
import heapq
import pandas as pd


In [ ]:

# 2020 population data
statePop = [
    5024279,733391,7151502,3011524,39538223,5773714,3605944,989948,21538187,10711908,
    1455271,1839106,12812508,6785528,3190369,2937880,4505836,4657757,1362359,6177224,
    7029917,10077331,5706494,2961279,6154913,1084225,1961504,3104614,1377529,9288994,
    2117522,20201249,10439388,779094,11799448,3959353,4237256,13002700,1097379,5118425,
    886667,6910840,29145505,3271616,643077,8631393,7705281,1793716,5893718,576851
]
stateNames = [
    "Alabama","Alaska","Arizona","Arkansas","California","Colorado","Connecticut","Delaware","Florida","Georgia",
    "Hawaii","Idaho","Illinois","Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland","Massachusetts",
    "Michigan","Minnesota","Mississippi","Missouri","Montana","Nebraska","Nevada","NewHampshire","NewJersey",
    "NewMexico","NewYork","NorthCarolina","NorthDakota","Ohio","Oklahoma","Oregon","Pennsylvania","RhodeIsland",
    "SouthCarolina","SouthDakota","Tennessee","Texas","Utah","Vermont","Virginia","Washington","WestVirginia",
    "Wisconsin","Wyoming"
]
name_fix = {
    "NewHampshire": "New Hampshire","NewJersey": "New Jersey","NewMexico": "New Mexico","NewYork": "New York",
    "NorthCarolina": "North Carolina","NorthDakota": "North Dakota","RhodeIsland": "Rhode Island",
    "SouthCarolina": "South Carolina","SouthDakota": "South Dakota","WestVirginia": "West Virginia"
}
stateNamesFixed = [name_fix.get(s, s) for s in stateNames]


In [ ]:

def hh_priority(pop, n):
    return pop / math.sqrt(n * (n + 1))

def clean_state_name(s):
    s = str(s)
    return s.replace("\u00a0", "").replace("\xa0", "").strip()


In [ ]:

TARGET_SEATS = 435
seats = [1] * 50
total = 50

heap = []
for i in range(50):
    p = statePop[i] / math.sqrt(2)
    heapq.heappush(heap, (-p, i))

while total < TARGET_SEATS:
    neg_p, i = heapq.heappop(heap)
    seats[i] += 1
    total += 1
    new_p = hh_priority(statePop[i], seats[i])
    heapq.heappush(heap, (-new_p, i))


In [ ]:

apportionment = pd.DataFrame({
    "State": stateNamesFixed,
    "Population_2020": statePop,
    "Representatives_Calc": seats
}).sort_values("State").reset_index(drop=True)
print("Total:", apportionment["Representatives_Calc"].sum())
apportionment.head()


In [ ]:

actual = pd.read_csv("StatesPopulation.csv")
actual["State"] = actual["State"].map(clean_state_name)
actual = actual.rename(columns={"Number of Voting Seats in Congress": "Representatives_Actual"})

merged = pd.merge(
    apportionment.rename(columns={"State": "StateKey"}),
    actual.rename(columns={"State": "StateKey"})[["StateKey","Representatives_Actual"]],
    on="StateKey", how="left"
)
merged["Match"] = merged["Representatives_Calc"] == merged["Representatives_Actual"]
print("Matches:", merged["Match"].sum(), "/ 50")
merged.head(10)


In [ ]:

assert merged["Representatives_Calc"].sum() == 435
assert merged["Representatives_Actual"].sum() == 435
assert merged["Match"].all()
print("✅ All checks passed. Ready to submit.")
